In [1]:
import datetime
import pandas as pd
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# Search for conjunctions

The cornerstone aspect of the AuroraX search engine is the ability to automatically find conjunctions between ground-based instruments and spacecrafts. One aspect that makes AuroraX's conjunction search different from other solution is that it will only find times where there is ground-based data available. It can also limit results to conjunctions where certain spacecraft instruments were operating. This integration with data availability saves us all valuable minutes and hours when surveying or searching for data that could be helpful for your next paper. 

A great way to get introduced to the AuroraX Conjunction Search capabilities is to head on over the [web page](https://aurorax.space/conjunctionSearch/standard) and play around with UI there. Nearly any request you can do in the web interface, you can also do using the below function.

In [18]:
# search for conjunctions between any THEMIS ASI and any Swarm spacecraft
#
# set up search parameters
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 1, 6, 59, 59)
ground = [{"programs": ["themis-asi"]}]
space = [{"programs": ["swarm"]}]
distance = 500

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=ground, space=space, verbose=True)

[2024-12-13 15:53:58.723365] Search object created
[2024-12-13 15:54:03.548250] Request submitted
[2024-12-13 15:54:03.548924] Request ID: 0a0bd280-e4b4-4041-9767-0e81d12329a0
[2024-12-13 15:54:03.548955] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/0a0bd280-e4b4-4041-9767-0e81d12329a0
[2024-12-13 15:54:03.548991] Waiting for data ...
[2024-12-13 15:54:07.582739] Data is now available
[2024-12-13 15:54:07.583099] Retrieving data ...
[2024-12-13 15:54:11.158479] Retrieved 376.7 kB of data containing 49 records


In [ ]:
# output data
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)

# let's show the first 10 conjunctions, sorted by 'start' time
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
43,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
15,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
39,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",195.554240,298.419001,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
12,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",282.877438,285.744881,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
29,nbtrace,2020-01-01T00:11:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",53.391843,441.814398,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:11:00
4,nbtrace,2020-01-01T00:12:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",89.436436,332.998863,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:13:00
36,nbtrace,2020-01-01T00:13:00,2020-01-01T00:14:00,"[DataSource(identifier=50, program='themis-asi...",215.337413,330.343161,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:13:00
10,nbtrace,2020-01-01T00:13:00,2020-01-01T00:15:00,"[DataSource(identifier=50, program='themis-asi...",105.345328,488.349867,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:15:00
16,nbtrace,2020-01-01T00:13:00,2020-01-01T00:13:00,"[DataSource(identifier=54, program='themis-asi...",401.883399,401.883399,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:13:00,2020-01-01T00:13:00
33,nbtrace,2020-01-01T00:44:00,2020-01-01T00:45:00,"[DataSource(identifier=48, program='themis-asi...",176.558673,365.537448,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:45:00,2020-01-01T00:44:00


# Search for conjunctions with metadata filters

Using the metadata filters to help search for conjunctions is one of the more advanced, and highly powerful, tools available. There are many metadata filters for spacecrafts, and some very useful ML-derived filters for ground-based all-sky imagers. 

Let's have a look at a simple example here, where we search for conjunctions filtering for when spacecrafts are in the north polar cap. The regions available to choose from are all directly pulled from SSCWeb. Almost all data used by AuroraX for spacecrafts is from SSCWeb without any alterations, only organization in the AuroraX database to enable the search engine to function.

If you're interested in interacting with the ML-derived metadata for ground-based instruments, have a look at [this crib sheet](https://github.com/aurorax-space/pyaurorax/tree/main/examples/notebooks/search/search_ephemeris_ml.ipynb).

In [ ]:
# search for conjunctions between any THEMIS-ASI or REGO instrument, and any Swarm
# instrument where the north B-trace region is 'north polar cap'.
#
# NOTE: this region metadata is not derived by AuroraX, but instead directly from
# SSCWeb. See the above text for more info.
#
# set up search paramters
start = datetime.datetime(2019, 2, 1, 0, 0, 0)
end = datetime.datetime(2019, 2, 10, 23, 59, 59)
ground = [{"programs": ["themis-asi", "rego"]}]
space = [{
    "programs": ["swarm"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [{
            "key": "nbtrace_region",
            "operator": "in",
            "values": ["north polar cap"]
        }]
    }
}]

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, ground=ground, space=space, verbose=True)

[2024-12-13 15:31:15.467590] Search object created
[2024-12-13 15:31:22.357711] Request submitted
[2024-12-13 15:31:22.357969] Request ID: 24949f54-7def-41ef-b6cf-e1f4c5ad8726
[2024-12-13 15:31:22.358047] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/24949f54-7def-41ef-b6cf-e1f4c5ad8726
[2024-12-13 15:31:22.358079] Waiting for data ...
[2024-12-13 15:31:28.703363] Data is now available
[2024-12-13 15:31:28.703534] Retrieving data ...
[2024-12-13 15:31:39.436258] Retrieved 2.1 MB of data containing 273 records


In [13]:
# view the first 10 conjunctions
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
95,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=53, program='themis-asi...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
200,nbtrace,2019-02-01T02:20:00,2019-02-01T02:21:00,"[DataSource(identifier=80, program='rego', pla...",153.987890,371.020379,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:20:00,2019-02-01T02:21:00
232,nbtrace,2019-02-01T02:21:00,2019-02-01T02:22:00,"[DataSource(identifier=83, program='rego', pla...",267.685472,300.984658,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T02:22:00,2019-02-01T02:21:00
168,nbtrace,2019-02-01T03:55:00,2019-02-01T03:56:00,"[DataSource(identifier=78, program='rego', pla...",471.096543,490.131048,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:56:00,2019-02-01T03:55:00
228,nbtrace,2019-02-01T03:57:00,2019-02-01T03:57:00,"[DataSource(identifier=83, program='rego', pla...",496.774661,496.774661,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T03:57:00,2019-02-01T03:57:00
254,nbtrace,2019-02-01T04:55:00,2019-02-01T04:55:00,"[DataSource(identifier=83, program='rego', pla...",438.433977,438.433977,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:55:00
147,nbtrace,2019-02-01T04:55:00,2019-02-01T04:56:00,"[DataSource(identifier=83, program='rego', pla...",424.997148,445.453544,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T04:55:00,2019-02-01T04:56:00
172,nbtrace,2019-02-01T05:31:00,2019-02-01T05:31:00,"[DataSource(identifier=78, program='rego', pla...",440.820767,440.820767,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T05:31:00,2019-02-01T05:31:00
258,nbtrace,2019-02-01T06:28:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",234.199376,484.697629,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:28:00
143,nbtrace,2019-02-01T06:29:00,2019-02-01T06:30:00,"[DataSource(identifier=83, program='rego', pla...",280.074450,466.631937,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-02-01T06:29:00,2019-02-01T06:30:00


# Multiple ground and space instruments, and advanced distances

Since the `ground` and `space` paramters are lists, we can search for conjunctions between multiple ground and space instruments. Each item of the lists are evaluated as a logical AND statement when searching for conjunctions. In the [web interface](https://aurorax.space/conjunctionSearch/standard), these are called 'criteria blocks' and are represented as each row in the table used for setting up a query.

You'll also notice that you can set distances between each list item (ground1-space1, ground1-space2, etc.). This can be quite powerful, but just remember to think about these values to ensure you're doing the right kind of distance evaluation between each. It can get confusing easily for each criteria block you add. A value of `None` will tell the search engine to not care about the distance between those two items.

In [ ]:
# search for conjunctions between any REGO instrument, any TREx instrument,
# any Swarm spacecraft, and any THEMIS spacecraft, up to 500 km apart for
# each ground instrument and spacecraft.
#
# since we have four list items, we can refer to this as a 'quadruple conjunction'
# search
#
# set up search parameters
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 4, 23, 59, 59)
ground = [{"programs": ["rego"]}, {"programs": ["trex"]}]
space = [{"programs": ["swarm"]}, {"programs": ["themis"]}]
advanced_distances = {
    "ground1-ground2": None,
    "ground1-space1": 500,
    "ground1-space2": 500,
    "ground2-space1": 500,
    "ground2-space2": 500,
    "space1-space2": None
}

# perform search
s = aurorax.search.conjunctions.search(start, end, advanced_distances, ground=ground, space=space, verbose=True)

[2024-12-13 15:47:49.268716] Search object created
[2024-12-13 15:47:56.005713] Request submitted
[2024-12-13 15:47:56.006295] Request ID: 776db61d-af7a-46d3-8c42-549c131d4e1b
[2024-12-13 15:47:56.006380] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/776db61d-af7a-46d3-8c42-549c131d4e1b
[2024-12-13 15:47:56.006435] Waiting for data ...
[2024-12-13 15:48:04.767364] Checking for data ...
[2024-12-13 15:48:17.727508] Data is now available
[2024-12-13 15:48:17.727899] Retrieving data ...
[2024-12-13 15:48:24.485981] Retrieved 87.8 kB of data containing 8 records


In [15]:
# show the results
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
4,nbtrace,2020-01-02T13:00:00,2020-01-02T13:01:00,"[DataSource(identifier=81, program='rego', pla...",384.347496,440.461162,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-02T13:01:00,2020-01-02T13:00:00
7,nbtrace,2020-01-02T13:00:00,2020-01-02T13:01:00,"[DataSource(identifier=81, program='rego', pla...",384.347496,440.461162,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-02T13:01:00,2020-01-02T13:00:00
0,nbtrace,2020-01-03T12:33:00,2020-01-03T12:35:00,"[DataSource(identifier=81, program='rego', pla...",111.324353,485.848555,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:35:00
1,nbtrace,2020-01-03T12:33:00,2020-01-03T12:35:00,"[DataSource(identifier=81, program='rego', pla...",111.324353,485.848555,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:35:00
3,nbtrace,2020-01-03T12:33:00,2020-01-03T12:34:00,"[DataSource(identifier=81, program='rego', pla...",122.118464,294.065540,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:33:00
6,nbtrace,2020-01-03T12:33:00,2020-01-03T12:34:00,"[DataSource(identifier=81, program='rego', pla...",122.118464,294.065540,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:34:00,2020-01-03T12:33:00
2,nbtrace,2020-01-03T12:40:00,2020-01-03T12:42:00,"[DataSource(identifier=81, program='rego', pla...",255.883772,476.507384,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:41:00,2020-01-03T12:40:00
5,nbtrace,2020-01-03T12:40:00,2020-01-03T12:42:00,"[DataSource(identifier=81, program='rego', pla...",255.883772,476.507384,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-03T12:41:00,2020-01-03T12:40:00


# Search for conjunctions between spacecrafts only

Since the conjunction search engine boils down to just finding matches between any two data sources, we can easily extend this to find conjunctions between only spacecrafts.

In [ ]:
# search for conjunctions between Swarm A or Swarm B, and any THEMIS
# spacecraft with the south B-trace region matching 'south polar cap'
#
# set up search parameters
start = datetime.datetime(2019, 1, 1, 0, 0, 0)
end = datetime.datetime(2019, 1, 1, 23, 59, 59)
space = [{
    "programs": ["themis"]
}, {
    "programs": ["swarm"],
    "platforms": ["swarma", "swarmb"],
    "hemisphere": ["southern"],
    "ephemeris_metadata_filters": {
        "logical_operator": "AND",
        "expressions": [{
            "key": "sbtrace_region",
            "operator": "in",
            "values": ["south polar cap"]
        }]
    }
}]
distance = 500

# perform search
s = aurorax.search.conjunctions.search(start, end, distance, space=space, verbose=True)

[2024-12-13 15:55:08.570697] Search object created
[2024-12-13 15:55:15.094347] Request submitted
[2024-12-13 15:55:15.094641] Request ID: bbff78af-70d4-46cc-9d9e-67de34b642fe
[2024-12-13 15:55:15.094691] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/bbff78af-70d4-46cc-9d9e-67de34b642fe
[2024-12-13 15:55:15.094720] Waiting for data ...
[2024-12-13 15:55:20.290931] Data is now available
[2024-12-13 15:55:20.291162] Retrieving data ...
[2024-12-13 15:55:26.752761] Retrieved 17.3 kB of data containing 2 records


In [22]:
# show the results
conjunctions = [c.__dict__ for c in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
1,nbtrace,2019-01-01T10:01:00,2019-01-01T10:01:00,"[DataSource(identifier=33, program='themis', p...",311.183877,311.183877,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-01-01T10:01:00,2019-01-01T10:01:00
0,nbtrace,2019-01-01T12:08:00,2019-01-01T12:08:00,"[DataSource(identifier=33, program='themis', p...",215.817544,215.817544,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2019-01-01T12:08:00,2019-01-01T12:08:00


# Explore results in Swarm-Aurora

Just like the web interface, you can also explore the results of a conjunction search in Swarm-Aurora. This is enabled by knowing the 'search request ID' value, and a handy helper function in PyAuroraX. The function will spawn a browser window and open up Swarm-Aurora, loading up the conjunction results that were found for that search request.

In [2]:
# we'll do a search between a couple THEMIS ASIs and any Swarm spacecrafts
start = datetime.datetime(2019, 1, 1, 0, 0, 0)
end = datetime.datetime(2019, 1, 3, 23, 59, 59)
ground_params = [{"programs": ["themis-asi"], "platforms": ["fort smith", "gillam"]}]
space_params = [{"programs": ["swarm"], "hemisphere": ["northern"]}]
distance = 500
s = aurorax.search.conjunctions.search(start=start, end=end, distance=distance, ground=ground_params, space=space_params, verbose=True)

[2024-12-13 16:27:43.166223] Search object created
[2024-12-13 16:27:51.789710] Request submitted
[2024-12-13 16:27:51.790398] Request ID: 72641199-69f4-41fa-9229-c02b88a2b8c6
[2024-12-13 16:27:51.790491] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/72641199-69f4-41fa-9229-c02b88a2b8c6
[2024-12-13 16:27:51.790530] Waiting for data ...
[2024-12-13 16:27:55.886382] Data is now available
[2024-12-13 16:27:55.886875] Retrieving data ...
[2024-12-13 16:27:59.339603] Retrieved 84.6 kB of data containing 11 records


In [3]:
# with the search now done and saved to the 's' variable, we can use that
# object with the helper function
#
# NOTE: some editors, such as VSCode, sometimes have issues figuring out the browser to
# use when opening Swarm-Aurora. If it helps, we can set the 'browser' parameter to the
# helper function with one of the available values specified in the official webbrowser
# module documentation: https://docs.python.org/3/library/webbrowser.html#webbrowser.get
#
# open browser to Swarm-Aurora for this search
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s)
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s, browser='google-chrome')

In [4]:
# alternatively, you can also just get the web URL for Swarm-Aurora for this
# specific request, and copy paste it into the browser manually.
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=72641199-69f4-41fa-9229-c02b88a2b8c6


# Do the search step-by-step

Under the hood, the AuroraX API performs a conjunction search asynchronously. Note that this does not mean that it can be done using a Python async method; it means that PyAuroraX does more than just a single HTTP request when doing a search. With the API operating this way, it adds some more complexity within PyAuroraX but also opens the search up to some very important capabilities. 

The main capability enabled by this architecture is being able to perform queries for large timeframes, and/or between a large number of data sources. Queries like this can easily take several minutes, and cause browsers and programmatic HTTP requests to timeout.

Instead of using the `aurorax.search.conjunctions.search()` method, you can also perform a conjunction search step-by-step if you want more control over the process. One use case for this is if you want to start a series of conjunction searches, and then go through each getting the results back as they finish, as opposed to doing one search at a time.

In [2]:
# set up the search parameters
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 1, 6, 59, 59)
ground = [{"programs": ["themis-asi"]}]
space = [{"programs": ["swarm"]}]
distance = 500

# create a Search object
s = pyaurorax.search.ConjunctionSearch(aurorax, start, end, distance, ground=ground, space=space)
s.pretty_print()

ConjunctionSearch:
  executed     : False
  completed    : False
  request_id   : 
  request      : None
  request_url  : 
  data_url     : 
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {}
  data         : 
  logs         : 


In [3]:
# submit the search to begin
s.execute()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : False
  request_id   : 2d7ff9a0-670f-4c2b-814a-99d54100319d
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d
  data_url     : 
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {}
  data         : [0 conjunction results]
  logs         : [0 log messages]


In [4]:
# update the search request status
s.update_status()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : True
  request_id   : 2d7ff9a0-670f-4c2b-814a-99d54100319d
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d
  data_url     : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d/data
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '2d7ff9a0-670f-4c2b-814a-99d54100319d', 'query...
  data         : [0 conjunction results]
  logs         : [13 log messages]


In [5]:
# wait for the data to be available
s.wait()
s.update_status()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : True
  request_id   : 2d7ff9a0-670f-4c2b-814a-99d54100319d
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d
  data_url     : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d/data
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '2d7ff9a0-670f-4c2b-814a-99d54100319d', 'query...
  data         : [0 conjunction results]
  logs         : [13 log messages]


In [6]:
# now that we know the request is complete, let's retrieve the data
s.get_data()
s.pretty_print()

ConjunctionSearch:
  executed     : True
  completed    : True
  request_id   : 2d7ff9a0-670f-4c2b-814a-99d54100319d
  request      : AuroraXAPIResponse [202] (Accepted)
  request_url  : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d
  data_url     : https://api.aurorax.space/api/v1/conjunctions/requests/2d7ff9a0-670f-4c2b-814a-99d54100319d/data
  query        : {'start': '2020-01-01T00:00:00', 'end': '2020-01-01T06:59:59', 'ground': [{'prog...
  status       : {'search_request': {'request_id': '2d7ff9a0-670f-4c2b-814a-99d54100319d', 'query...
  data         : [49 conjunction results]
  logs         : [13 log messages]


In [7]:
# show the first 10 conjunction results 
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
43,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
15,nbtrace,2020-01-01T00:08:00,2020-01-01T00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:09:00,2020-01-01T00:08:00
39,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",195.554240,298.419001,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
12,nbtrace,2020-01-01T00:10:00,2020-01-01T00:11:00,"[DataSource(identifier=51, program='themis-asi...",282.877438,285.744881,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:10:00,2020-01-01T00:11:00
29,nbtrace,2020-01-01T00:11:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",53.391843,441.814398,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:11:00
4,nbtrace,2020-01-01T00:12:00,2020-01-01T00:13:00,"[DataSource(identifier=46, program='themis-asi...",89.436436,332.998863,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:12:00,2020-01-01T00:13:00
36,nbtrace,2020-01-01T00:13:00,2020-01-01T00:14:00,"[DataSource(identifier=50, program='themis-asi...",215.337413,330.343161,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:13:00
10,nbtrace,2020-01-01T00:13:00,2020-01-01T00:15:00,"[DataSource(identifier=50, program='themis-asi...",105.345328,488.349867,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:14:00,2020-01-01T00:15:00
16,nbtrace,2020-01-01T00:13:00,2020-01-01T00:13:00,"[DataSource(identifier=54, program='themis-asi...",401.883399,401.883399,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:13:00,2020-01-01T00:13:00
33,nbtrace,2020-01-01T00:44:00,2020-01-01T00:45:00,"[DataSource(identifier=48, program='themis-asi...",176.558673,365.537448,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01T00:45:00,2020-01-01T00:44:00
